In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import TruncatedSVD
import cluster_class
import LogisticRegression
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression as sklogi
import pickle

In [28]:
df = pd.read_csv("train.csv")
df.head()

,sentence,author
0,"[eugenie, grandet, by, honore, de, balzac, tra...","Balzac, Honoré de"
1,"[may, your, name,, that, of, one, whose, portr...","Balzac, Honoré de"
2,"[de, balzac.]","Balzac, Honoré de"
3,"[eugenie, grandet, i, there, are, houses, in, ...","Balzac, Honoré de"
4,"[within, these, houses, there, is,, perhaps,, ...","Balzac, Honoré de"


In [33]:
lines = cluster_class.clean_text(df['text'])
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(lines)]

model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# model = Word2Vec(lines, size=20, window=3, min_count=1, workers=4, iter=10)
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

class Case(cluster_class.Example):
    pass

examples = []
for _id, line, author in zip(df.index, lines, df['author']):
    examples.append(Case(_id, model.infer_vector(line), author, line))

train, test = LogisticRegression.split80_20(examples)


KeyboardInterrupt: 

In [4]:
print(train[700])

id19561:[ 0.08078597  0.25492775  0.17742518 -0.39117828  0.0727841 ]:[0, 0, 1]


In [5]:
train, test = LogisticRegression.split80_20(examples)

In [6]:
lg = sklogi().fit(np.array([list(i.getFeatures()) for i in train]), np.array([i.getLabel().index(1) for i in train]))

In [7]:
right, wrong = 0, 0
for i in range(len(test)):
    if lg.predict([list(test[i].getFeatures())])[0] == test[i].getLabel().index(1):
        right += 1
    else:
        wrong += 1
right/wrong # accuracy

0.6560913705583756